In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from Artha.nlp_extraction import run_pipeline, save_backup, load_backup, to_doc_dict
import Artha.data_process as dp
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import json
import numpy as np
np.set_printoptions(suppress=True)
from datetime import datetime, timedelta
from Artha.mentions import *
from collections import Counter
import pandas as pd

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')


CPU times: user 2.06 s, sys: 355 ms, total: 2.41 s
Wall time: 3.87 s


In [2]:
follow_weight = .5
print(len(neo.get_nodes()), len(neo.get_relations()))
# without mentions 711 6958

7 15


In [3]:
neo.clear_nodes()

In [4]:
%%time
# add follows between only people I follow, can change by getting rid of second 2 last line for anyone a person follows

# all_people = [i[1:-4] for i in os.listdir("../data/follows/")]
# checkra_people = [c[3] for c in dp.load_following("checkra_") ]


selected_people = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
selected_nodes = [c for c in dp.load_following("checkra_") if c[3] in selected_people]

selected_follows = dp.follow_edges(selected_people, follow_weight)

CPU times: user 7.53 ms, sys: 3.08 ms, total: 10.6 ms
Wall time: 12.1 ms


In [5]:
%%time
# print(len(follows))
neo.clear_nodes()

neo.load_follow_nodes(selected_nodes)
neo.load_follow_nodes(selected_follows)
neo.load_follow_relations(selected_follows)

print(len(neo.get_nodes()), len(neo.get_relations()))


7 15
CPU times: user 7.84 ms, sys: 2.6 ms, total: 10.4 ms
Wall time: 92.5 ms


In [6]:
%%time
# load all tweets
# all_tweets = []
# for ind, username in tqdm(enumerate(selected_people)):
#     cur_tweets = dp.load_tweets(username, location = "../data/tweets1/u")
#     all_tweets.extend([tweet for tweet in dp.clean_tweets(cur_tweets, username)])

# print("\n", len(all_tweets))
# #process all tweets at once
# docs = run_pipeline(all_tweets)
# 

docs = load_backup()
doc_dict = to_doc_dict(docs)

loaded
100%|██████████| 14474/14474 [00:00<00:00, 173547.15it/s]CPU times: user 5.23 s, sys: 126 ms, total: 5.35 s
Wall time: 5.35 s



In [26]:
%%time
# code for multiple graphs over time
day_scores = {}
date, window_count = "02/24/2021 00:00:00", 3

dates = [from_datetime(to_datetime(date)+timedelta(days=i)) for i in range(window_count)]

for i in tqdm(dates):
    all_mentions = [edge for person in selected_people 
                    for edge in get_mention_edges(
                                      doc_dict[person], person,
                                      follow_weight = follow_weight,
                                      win_start_date = date)]
    neo.clear_nodes("Coin")
    neo.load_mentions(all_mentions)
    day_scores[i] = neo.pagerank_scores()


all_ticks = list(set([i[0] for day in dates for i in day_scores[day]]))

tick_dict = {tick:np.zeros(window_count) for tick in all_ticks}

for ind, day in enumerate(day_scores.keys()):
    for score in day_scores[day]:
        tick_dict[score[0]][ind] = score[1]

pdpagerank = pd.DataFrame(tick_dict, index = pd.to_datetime(dates))

100%|██████████| 3/3 [00:01<00:00,  1.61it/s]CPU times: user 1.35 s, sys: 12.3 ms, total: 1.36 s
Wall time: 1.86 s



In [28]:
pdpagerank

,DOGE,SUSHI,GO,SUPER,AST,FIL,ALGO,ALPHA,ATOM,BNB,...,DENT,MATIC,YFI,USDC,TRY,NEO,SNT,ONE,WRX,BZRX
2021-02-24,1.043454,0.408526,0.19131,0.161961,0.164978,0.237497,0.55565,0.24166,0.165467,0.543995,...,0.252061,0.183298,0.465163,0.334849,0.176428,0.248019,0.153073,0.240342,0.212675,0.15367
2021-02-25,1.043454,0.408526,0.19131,0.161961,0.164978,0.237497,0.55565,0.24166,0.165467,0.543995,...,0.252061,0.183298,0.465163,0.334849,0.176428,0.248019,0.153073,0.240342,0.212675,0.15367
2021-02-26,1.043454,0.408526,0.19131,0.161961,0.164978,0.237497,0.55565,0.24166,0.165467,0.543995,...,0.252061,0.183298,0.465163,0.334849,0.176428,0.248019,0.153073,0.240342,0.212675,0.15367


In [10]:
# exchange tokens might have more mentions
# factor in mentions of coins in threads, so it gets more mentions than just 1